In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

import matplotlib.pyplot as plt
import numpy as np

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

AUTOTUNE = tf.data.experimental.AUTOTUNE
    
print(tf.__version__)


#test lalalalalalalala

In [ ]:
#test lalalalalalalala

HEIGHT = 256
WIDTH = 256
CHANNELS = 3
BATCH_SIZE = 1
OUTPUT_CHANNELS = 3
IMAGE_SIZE = [WIDTH, HEIGHT]
WINDOW_SIZE = 4

monet_adress = "../input/tif-project-data/Albrecht_Durer_256.tfrecords"
photo_adress = "../input/gan-getting-started/photo_tfrec/*.tfrec"

learning_rate1 = 2e-4
initial_learning_rate2 = 1e-4
end_learning_rate2 = 2e-5
learning_rate3 = 2e-5

steps_per_epoch = 1000
epochs1 = 1 #6
epochs2 = 0 #21 
epochs3 = 0 #4
repeat_factor = 15


In [ ]:
#Load data and  display  

#images(string) -> images(tensor)  
def decode_image(image):
    image = tf.image.decode_jpeg(image, CHANNELS)
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    image = tf.reshape(image, [*IMAGE_SIZE, CHANNELS])
    return image

#tfrec -> images(string)  
def read_tfrecord(example):
    tfrecord_format = {
        "image": tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    return image

#filenamn -> tfrec
def load_dataset(filenames, labeled=True, ordered=False):
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTOTUNE)
    return dataset

MONET_FILENAMES = tf.io.gfile.glob(monet_adress)
print('Monet TFRecord Files:', len(MONET_FILENAMES))

PHOTO_FILENAMES = tf.io.gfile.glob(str(photo_adress))
print('Photo TFRecord Files:', len(PHOTO_FILENAMES))

monet_ds = load_dataset(MONET_FILENAMES).batch(1)
photo_ds = load_dataset(PHOTO_FILENAMES).batch(1)
example_monet = next(iter(monet_ds))
example_photo = next(iter(photo_ds))

plt.subplot(121)
plt.title('Photo')
plt.imshow(example_photo[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('Monet')
plt.imshow(example_monet[0] * 0.5 + 0.5)


In [ ]:
# reduces the 2D dimensions by the stride.
def downsample(filters, size, apply_instancenorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    result = keras.Sequential()
    result.add(layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))

    if apply_instancenorm:
        result.add(tfa.layers.InstanceNormalization(gamma_initializer=gamma_init))

    result.add(layers.LeakyReLU())
    return result

def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    result = keras.Sequential()
    result.add(layers.Conv2DTranspose(filters, size, strides=2,
                                      padding='same',
                                      kernel_initializer=initializer,
                                      use_bias=False))

    result.add(tfa.layers.InstanceNormalization(gamma_initializer=gamma_init))

    if apply_dropout:
        result.add(layers.Dropout(0.5))

    result.add(layers.ReLU())

    return result

In [ ]:
def Generator():
    inputs = layers.Input(shape=[HEIGHT,WIDTH,CHANNELS])

    # bs = batch size
    down_stack = [
        downsample(64, WINDOW_SIZE, apply_instancenorm=False), # (bs, 128, 128, 64)
        downsample(128, WINDOW_SIZE), # (bs, 64, 64, 128)
        downsample(256, WINDOW_SIZE), # (bs, 32, 32, 256)
        downsample(512, WINDOW_SIZE), # (bs, 16, 16, 512)
        downsample(512, WINDOW_SIZE), # (bs, 8, 8, 512)
        downsample(512, WINDOW_SIZE), # (bs, 4, 4, 512)
        downsample(512, WINDOW_SIZE), # (bs, 2, 2, 512)
        downsample(512, WINDOW_SIZE), # (bs, 1, 1, 512)
    ]

    up_stack = [
        upsample(512, WINDOW_SIZE, apply_dropout=True), # (bs, 2, 2, 1024)
        upsample(512, WINDOW_SIZE, apply_dropout=True), # (bs, 4, 4, 1024)
        upsample(512, WINDOW_SIZE, apply_dropout=True), # (bs, 8, 8, 1024)
        upsample(512, WINDOW_SIZE), # (bs, 16, 16, 1024)
        upsample(256, WINDOW_SIZE), # (bs, 32, 32, 512)
        upsample(128, WINDOW_SIZE), # (bs, 64, 64, 256)
        upsample(64, WINDOW_SIZE), # (bs, 128, 128, 128)
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    last = layers.Conv2DTranspose(CHANNELS, WINDOW_SIZE,
                                  strides=2,
                                  padding='same',
                                  kernel_initializer=initializer,
                                  activation='tanh') # (bs, 256, 256, 3)

    x = inputs

    # Downsampling through the model
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = layers.Concatenate()([x, skip])

    x = last(x)

    return keras.Model(inputs=inputs, outputs=x)

In [ ]:
def Discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    inp = layers.Input(shape=[HEIGHT,WIDTH,CHANNELS], name='input_image')

    x = inp

    down1 = downsample(64, WINDOW_SIZE, False)(x) # (bs, 128, 128, 64)
    down2 = downsample(128, WINDOW_SIZE)(down1) # (bs, 64, 64, 128)
    down3 = downsample(256, WINDOW_SIZE)(down2) # (bs, 32, 32, 256)

    zero_pad1 = layers.ZeroPadding2D()(down3) # (bs, 34, 34, 256)
    conv = layers.Conv2D(512, WINDOW_SIZE, strides=1,
                         kernel_initializer=initializer,
                         use_bias=False)(zero_pad1) # (bs, 31, 31, 512)

    norm1 = tfa.layers.InstanceNormalization(gamma_initializer=gamma_init)(conv)

    leaky_relu = layers.LeakyReLU()(norm1)

    zero_pad2 = layers.ZeroPadding2D()(leaky_relu) # (bs, 33, 33, 512)

    last = layers.Conv2D(1, WINDOW_SIZE, strides=1,
                         kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

    return tf.keras.Model(inputs=inp, outputs=last)

In [ ]:
class CycleGan(keras.Model):
    def __init__(
        self,
        monet_generator,
        photo_generator,
        monet_discriminator,
        photo_discriminator,
        example_photo,
        loss_obj,
        lambda_cycle= tf.Variable(8.0),
        lambda_id = tf.Variable(1.)
        
        ):
        super(CycleGan, self).__init__()
        self.m_gen = monet_generator
        self.p_gen = photo_generator
        self.m_disc = monet_discriminator
        self.p_disc = photo_discriminator
        self.e_photo = example_photo
        self.loss_obj = loss_obj
        self.lambda_cycle = lambda_cycle
        self.lambda_id = lambda_id
        
    def compile(
        self,
        m_gen_optimizer,
        p_gen_optimizer,
        m_disc_optimizer,
        p_disc_optimizer,
        gen_loss_fn,
        disc_loss_fn,
        cycle_loss_fn,
        identity_loss_fn,
        diff_aug, 
        
    ):
        super(CycleGan, self).compile()
        self.m_gen_optimizer = m_gen_optimizer
        self.p_gen_optimizer = p_gen_optimizer
        self.m_disc_optimizer = m_disc_optimizer
        self.p_disc_optimizer = p_disc_optimizer
        self.gen_loss_fn = gen_loss_fn
        self.disc_loss_fn = disc_loss_fn
        self.cycle_loss_fn = cycle_loss_fn
        self.identity_loss_fn = identity_loss_fn
        self.diff_aug = diff_aug
    
    def train_step(self, batch_data):
        real_monet, real_photo = batch_data
        batch_size = tf.shape(real_monet)[0]
        with tf.GradientTape(persistent=True) as tape:
            
            # photo to monet back to photo
            fake_monet = self.m_gen(real_photo, training=True)
            cycled_photo = self.p_gen(fake_monet, training=True)

            # monet to photo back to monet
            fake_photo = self.p_gen(real_monet, training=True)
            cycled_monet = self.m_gen(fake_photo, training=True)

            # generating itself
            same_monet = self.m_gen(real_monet, training=True)
            same_photo = self.p_gen(real_photo, training=True)
            
            # Diffaugment monet and photo
            both_monet = tf.concat([real_monet, fake_monet], axis=0)            
            aug_monet = self.diff_aug(both_monet)
            aug_real_monet = aug_monet[:batch_size]
            aug_fake_monet = aug_monet[batch_size:]
            
            
            both_photo = tf.concat([real_photo, fake_photo], axis=0)            
            aug_photo = self.diff_aug(both_photo)
            aug_real_photo = aug_photo[:batch_size]
            aug_fake_photo = aug_photo[batch_size:]
            
            # discriminator used to check, inputing real images
            disc_real_monet = self.m_disc(aug_real_monet, training=True)
            disc_real_photo = self.p_disc(aug_real_photo, training=True)

            # discriminator used to check, inputing fake images
            disc_fake_monet = self.m_disc(aug_fake_monet, training=True)
            disc_fake_photo = self.p_disc(aug_fake_photo, training=True)

            # evaluates generator loss
            monet_gen_loss = self.gen_loss_fn(disc_fake_monet, self.loss_obj)
            photo_gen_loss = self.gen_loss_fn(disc_fake_photo, self.loss_obj)

            # evaluates total cycle consistency loss
            total_cycle_loss = self.cycle_loss_fn(real_monet, cycled_monet, self.lambda_cycle) + self.cycle_loss_fn(real_photo, cycled_photo, self.lambda_cycle)
            #evaluate identity loss
            monet_idloss = self.identity_loss_fn(real_monet, same_monet, self.lambda_id)
            photo_idloss = self.identity_loss_fn(real_photo, same_photo, self.lambda_id)
            # evaluates total generator loss
            total_monet_gen_loss = monet_gen_loss + total_cycle_loss + monet_idloss
            total_photo_gen_loss = photo_gen_loss + total_cycle_loss + photo_idloss

            # evaluates discriminator loss  
            monet_disc_loss = self.disc_loss_fn(disc_real_monet, disc_fake_monet, self.loss_obj)
            photo_disc_loss = self.disc_loss_fn(disc_real_photo, disc_fake_photo, self.loss_obj)

        # Calculate the gradients for generator and discriminator
        monet_generator_gradients = tape.gradient(total_monet_gen_loss,
                                                  self.m_gen.trainable_variables)
        photo_generator_gradients = tape.gradient(total_photo_gen_loss,
                                                  self.p_gen.trainable_variables)

        monet_discriminator_gradients = tape.gradient(monet_disc_loss,
                                                      self.m_disc.trainable_variables)
        photo_discriminator_gradients = tape.gradient(photo_disc_loss,
                                                      self.p_disc.trainable_variables)

        # Apply the gradients to the optimizer
        self.m_gen_optimizer.apply_gradients(zip(monet_generator_gradients,
                                                 self.m_gen.trainable_variables))

        self.p_gen_optimizer.apply_gradients(zip(photo_generator_gradients,
                                                 self.p_gen.trainable_variables))

        self.m_disc_optimizer.apply_gradients(zip(monet_discriminator_gradients,
                                                  self.m_disc.trainable_variables))

        self.p_disc_optimizer.apply_gradients(zip(photo_discriminator_gradients,
                                                  self.p_disc.trainable_variables))
        
        return {
            "monet_gen_loss": total_monet_gen_loss,
            "photo_gen_loss": total_photo_gen_loss,
            "monet_disc_loss": monet_disc_loss,
            "photo_disc_loss": photo_disc_loss,
            "total_cycle_loss": total_cycle_loss,
            "monet_gen_loss_disc": monet_gen_loss,
            "monet_id_loss": monet_idloss
        }

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        new_val=tf.math.scalar_mul(0.7,self.model.lambda_cycle)
        val = tf.math.maximum(tf.Variable(0.0005), new_val)
        self.model.lambda_cycle.assign(val)  
        
        new_id=tf.math.scalar_mul(0.7,self.model.lambda_id)
        new_id = tf.math.maximum(new_id, tf.Variable(0.005))
        self.model.lambda_id.assign(new_id)
        
    def on_epoch_end(self, epoch, logs=None):
        image = self.model.m_gen(self.model.e_photo)
        plt.figure(figsize=(6,6))
        plt.title("Monet-esque Photo")
        plt.imshow(image[0] * 0.5 + 0.5)
        plt.show()

In [ ]:
with strategy.scope():
  def discriminator_loss(real, generated, loss_obj):
    ones = tf.ones_like(real)
    smooth = tf.random.uniform([1],0.9,1)
    ones = tf.math.scalar_mul(smooth[0], ones)
    zeros = tf.zeros_like(real)

    real_loss = loss_obj(ones, real)
    generated_loss = loss_obj(zeros, generated)

    total_disc_loss = real_loss + generated_loss
    return total_disc_loss * 0.5

with strategy.scope():
  def generator_loss(generated, loss_obj):
    return loss_obj(tf.ones_like(generated), generated)

with strategy.scope():
  def calc_cycle_loss(real_image, cycled_image, LAMBDA):
    loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
    return LAMBDA * loss1

with strategy.scope():
  def identity_loss(real_image, same_image, LAMBDA):
    loss = tf.reduce_mean(tf.abs(real_image - same_image))
    return LAMBDA * 0.5 * loss

In [ ]:
# Differentiable Augmentation taken from: https://github.com/mit-han-lab/data-efficient-gans/blob/master/DiffAugment_tf.py
with strategy.scope():
    def DiffAugment(x, policy='color,translation,cutout', channels_first=False):
        if policy:
            if channels_first:
                x = tf.transpose(x, [0, 2, 3, 1])
            for p in policy.split(','):
                for f in AUGMENT_FNS[p]:
                    x = f(x)
            if channels_first:
                x = tf.transpose(x, [0, 3, 1, 2])
        return x

    def rand_brightness(x):
        magnitude = tf.random.uniform([tf.shape(x)[0], 1, 1, 1]) - 0.5
        x = x + magnitude
        return x


    def rand_saturation(x):
        magnitude = tf.random.uniform([tf.shape(x)[0], 1, 1, 1]) * 2
        x_mean = tf.reduce_mean(x, axis=3, keepdims=True)
        x = (x - x_mean) * magnitude + x_mean
        return x


    def rand_contrast(x):
        magnitude = tf.random.uniform([tf.shape(x)[0], 1, 1, 1]) + 0.5
        x_mean = tf.reduce_mean(x, axis=[1, 2, 3], keepdims=True)
        x = (x - x_mean) * magnitude + x_mean
        return x


    def rand_translation(x, ratio=0.125):
        batch_size = tf.shape(x)[0]
        image_size = tf.shape(x)[1:3]
        shift = tf.cast(tf.cast(image_size, tf.float32) * ratio + 0.5, tf.int32)
        translation_x = tf.random.uniform([batch_size, 1], -shift[0], shift[0] + 1, dtype=tf.int32)
        translation_y = tf.random.uniform([batch_size, 1], -shift[1], shift[1] + 1, dtype=tf.int32)
        grid_x = tf.clip_by_value(tf.expand_dims(tf.range(image_size[0], dtype=tf.int32), 0) + translation_x + 1, 0, image_size[0] + 1)
        grid_y = tf.clip_by_value(tf.expand_dims(tf.range(image_size[1], dtype=tf.int32), 0) + translation_y + 1, 0, image_size[1] + 1)
        x = tf.gather_nd(tf.pad(x, [[0, 0], [1, 1], [0, 0], [0, 0]]), tf.expand_dims(grid_x, -1), batch_dims=1)
        x = tf.transpose(tf.gather_nd(tf.pad(tf.transpose(x, [0, 2, 1, 3]), [[0, 0], [1, 1], [0, 0], [0, 0]]), tf.expand_dims(grid_y, -1), batch_dims=1), [0, 2, 1, 3])
        return x


    def rand_cutout(x, ratio=0.5):
        batch_size = tf.shape(x)[0]
        image_size = tf.shape(x)[1:3]
        cutout_size = tf.cast(tf.cast(image_size, tf.float32) * ratio + 0.5, tf.int32)
        offset_x = tf.random.uniform([tf.shape(x)[0], 1, 1], maxval=image_size[0] + (1 - cutout_size[0] % 2), dtype=tf.int32)
        offset_y = tf.random.uniform([tf.shape(x)[0], 1, 1], maxval=image_size[1] + (1 - cutout_size[1] % 2), dtype=tf.int32)
        grid_batch, grid_x, grid_y = tf.meshgrid(tf.range(batch_size, dtype=tf.int32), tf.range(cutout_size[0], dtype=tf.int32), tf.range(cutout_size[1], dtype=tf.int32), indexing='ij')
        cutout_grid = tf.stack([grid_batch, grid_x + offset_x - cutout_size[0] // 2, grid_y + offset_y - cutout_size[1] // 2], axis=-1)
        mask_shape = tf.stack([batch_size, image_size[0], image_size[1]])
        cutout_grid = tf.maximum(cutout_grid, 0)
        cutout_grid = tf.minimum(cutout_grid, tf.reshape(mask_shape - 1, [1, 1, 1, 3]))
        mask = tf.maximum(1 - tf.scatter_nd(cutout_grid, tf.ones([batch_size, cutout_size[0], cutout_size[1]], dtype=tf.float32), mask_shape), 0)
        x = x * tf.expand_dims(mask, axis=3)
        return x


    AUGMENT_FNS = {
        'color': [rand_brightness, rand_saturation, rand_contrast],
        'translation': [rand_translation],
        'cutout': [rand_cutout],
    }

# **TRAINING **

In [ ]:
with strategy.scope():
    monet_generator = Generator() # transforms photos to Monet-esque paintings
    photo_generator = Generator() # transforms Monet paintings to be more like photos

    monet_discriminator = Discriminator() # differentiates real Monet paintings and generated Monet paintings
    photo_discriminator = Discriminator() # differentiates real photos and generated photos

In [ ]:
with strategy.scope():
    monet_generator_optimizer = tf.keras.optimizers.Adam(learning_rate1, beta_1=0.5)
    photo_generator_optimizer = tf.keras.optimizers.Adam(learning_rate1, beta_1=0.5)

    monet_discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate1, beta_1=0.5)
    photo_discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate1, beta_1=0.5)
   
    cycle_gan_model = CycleGan(
        monet_generator, photo_generator, monet_discriminator, photo_discriminator, example_photo, 
        tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
    )

    cycle_gan_model.compile(
        m_gen_optimizer = monet_generator_optimizer,
        p_gen_optimizer = photo_generator_optimizer,
        m_disc_optimizer = monet_discriminator_optimizer,
        p_disc_optimizer = photo_discriminator_optimizer,
        gen_loss_fn = generator_loss,
        disc_loss_fn = discriminator_loss,
        cycle_loss_fn = calc_cycle_loss,
        identity_loss_fn = identity_loss,
        diff_aug = DiffAugment,
    )

    my_dataset = tf.data.Dataset.zip((monet_ds, photo_ds)) 
    my_dataset = my_dataset.repeat(epochs1*repeat_factor)
    cycle_gan_model.fit(
        my_dataset, steps_per_epoch=steps_per_epoch, epochs=epochs1, callbacks=[CustomCallback()]
    )

In [ ]:
with strategy.scope():
    decay_steps = epochs2*steps_per_epoch
    lr_schedule_g = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate2, decay_steps, end_learning_rate2, power=1,
        cycle=False, name=None
    )
    lr_schedule_d = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate2, decay_steps, end_learning_rate2, power=1,
        cycle=False, name=None
    )

    monet_generator_optimizer = tf.keras.optimizers.Adam(lr_schedule_g, beta_1=0.5)
    photo_generator_optimizer = tf.keras.optimizers.Adam(lr_schedule_g, beta_1=0.5)

    monet_discriminator_optimizer = tf.keras.optimizers.Adam(lr_schedule_d, beta_1=0.5)
    photo_discriminator_optimizer = tf.keras.optimizers.Adam(lr_schedule_d, beta_1=0.5)

    cycle_gan_model.compile(
        m_gen_optimizer = monet_generator_optimizer,
        p_gen_optimizer = photo_generator_optimizer,
        m_disc_optimizer = monet_discriminator_optimizer,
        p_disc_optimizer = photo_discriminator_optimizer,
        gen_loss_fn = generator_loss,
        disc_loss_fn = discriminator_loss,
        cycle_loss_fn = calc_cycle_loss,
        identity_loss_fn = identity_loss,
        diff_aug = DiffAugment,
    )
    my_dataset = tf.data.Dataset.zip((monet_ds, photo_ds)) 
    my_dataset = my_dataset.repeat(epochs2*repeat_factor)
    
    cycle_gan_model.fit(
        my_dataset,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs2,
        callbacks=[CustomCallback()]
    )

In [ ]:
with strategy.scope():
    monet_generator_optimizer = tf.keras.optimizers.Adam(learning_rate3, beta_1=0.5)
    photo_generator_optimizer = tf.keras.optimizers.Adam(learning_rate3, beta_1=0.5)

    monet_discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate3, beta_1=0.5)
    photo_discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate3, beta_1=0.5)
   
    cycle_gan_model = CycleGan(
        monet_generator, photo_generator, monet_discriminator, photo_discriminator, example_photo, 
        tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
    )

    cycle_gan_model.compile(
        m_gen_optimizer = monet_generator_optimizer,
        p_gen_optimizer = photo_generator_optimizer,
        m_disc_optimizer = monet_discriminator_optimizer,
        p_disc_optimizer = photo_discriminator_optimizer,
        gen_loss_fn = generator_loss,
        disc_loss_fn = discriminator_loss,
        cycle_loss_fn = calc_cycle_loss,
        identity_loss_fn = identity_loss,
        diff_aug = DiffAugment,
    )

    my_dataset = tf.data.Dataset.zip((monet_ds, photo_ds)) 
    my_dataset = my_dataset.repeat(epochs3*repeat_factor)
    cycle_gan_model.fit(
        my_dataset, steps_per_epoch=steps_per_epoch, epochs=epochs3, callbacks=[CustomCallback()]
    )
    monet_generator.save("/kaggle/working/monet_generator_3.h5")
    photo_generator.save("/kaggle/working/photo_generator_3.h5")

In [ ]:
for img in photo_ds.take(5):
    prediction = monet_generator(img, training=False)
    anti_prediction = photo_generator(prediction, training=False)
    for i in range(prediction.shape[0]):
        plt.figure(figsize=(12,12))

        plt.subplot(1, 3, 1)
        plt.imshow(img[i] * 0.5 + 0.5)

        plt.subplot(1, 3, 2)
        plt.imshow(prediction[i] * 0.5 + 0.5)
        
        plt.subplot(1, 3, 3)
        plt.imshow(anti_prediction[i] * 0.5 + 0.5)

        plt.axis('off')

In [ ]:
from IPython.display import FileLink 
%cd /kaggle/working
FileLink("photo_generator_3.h5")
#FileLink("/kaggle/working/photo_generator_3.h5")